<a href="https://colab.research.google.com/github/Saifullah785/machine-learning-engineer-roadmap/blob/main/Lecture_28_columntransformer_in_machine_learning/Lecture_28_columntransformer_in_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture: 28 ColumnTransformer in Machine Learning**


---

**🔧 How to Use ColumnTransformer in Scikit-learn**


**✅ What is ColumnTransformer?**


---


The ColumnTransformer is a powerful tool in scikit-learn used when:

You want to apply different preprocessing to different columns.

You work with both numeric and categorical data.

You want a clean pipeline for modeling.

# **🔧 Real-World Analogy**


---



**Imagine you're running a factory:**

You wash clothes in one machine (numerical scaling).

You iron clothes in another (categorical encoding).

You need a controller to send each item to the correct machine.
That controller is the ColumnTransformer.

**💼 When Do We Use It?**

Preprocessing mixed-type data (e.g., numerical + categorical).

Use inside ML Pipelines.

Applying multiple transformers in parallel.

In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('/content/covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [8]:
# df['city'].value_counts()
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


In [9]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [19]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)


In [20]:
x_train.head()

,age,gender,fever,cough,city
60,24,Female,102.0,Strong,Bangalore
28,16,Male,104.0,Mild,Kolkata
81,65,Male,99.0,Mild,Delhi
91,38,Male,NaN,Mild,Delhi
96,51,Female,101.0,Strong,Kolkata


# **1.Aam Zindagi**

In [21]:
# all missing value is replaced with mean of data
# adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape

(80, 1)

In [22]:
# Ordinalencoding ->cough

oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

#also the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [29]:
# OneHotEncoding -> gender,city

ohe = OneHotEncoder(drop='first')
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']]).toarray()

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']]).toarray()

x_train_gender_city.shape

(80, 4)

In [32]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [35]:
# concatenate all encoded columns

x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)

# also the test data

x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

# x_train_transformed
x_train_transformed.shape


(80, 7)

# **Mentos Zindagi**

In [36]:
from sklearn.compose import ColumnTransformer

In [39]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [42]:
transformer.fit_transform(x_train).shape
# transformer.fit_transform(x_train)

(80, 7)

In [44]:
# transformer.fit_transform(x_test).shape
transformer.fit_transform(x_test)

array([[100.52941176,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [100.52941176,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  84.        ],
       [102.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [103.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  73.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.    